## JG Thoughts 
- Unemployment rate vs employment #s? Look at change in UE rate or total as of certain point in time
- Inc_Per_CAp_2022 -> typo?
- Population Density per mi² -> what yr? Also is string
- NA values in NJ
- Convert weeks into %?

- Maybe PCA for the white, black, Hispanic stats? But missing a decent chunk it looks like. Maybe only black or white. But a lot of 0s so maybe just exclde entirely  
- Maybe do something to 'Population' like catgy 1-5 for large small or medium. But can leave for now I guess 

- Do SVM or NN models? SVM mostly for classification. NN maybe not enough data

## Data Cleaning & Preprocessing

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.ensemble import VotingRegressor

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor


In [369]:
df = pd.read_csv('Data/Cleaned/county_df.csv')
df.head()

,FIPS_x,County,Years of Potential Life Lost Rate (premature death),YPLL Rate (Black),YPLL Rate (Hispanic),YPLL Rate (White),% Fair/Poor Health,percent_smokers,percent_obese,Food Environment Index,...,cases_2020,cases_2021,cases_2022,deaths_2020,deaths_2021,deaths_2022,Masks,FIPS_y,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_65PlusPop_Pct
0,1001,Autauga,8824.0,10471.0,NaN,8707.0,18,19,38,7.2,...,4190.0,11018.0,18961.0,5631.0,47405.0,124934.0,267.0,1001,42.2,73.8
1,1003,Baldwin,7225.0,10042.0,3087.0,7278.0,18,17,31,8.0,...,13601.0,39911.0,67496.0,12412.0,148723.0,397246.0,267.0,1003,53.2,89.9
2,1005,Barbour,9586.0,11333.0,NaN,7310.0,26,22,44,5.6,...,1514.0,3860.0,7027.0,2035.0,24364.0,60044.0,267.0,1005,44.5,75.3
3,1007,Bibb,11784.0,14813.0,NaN,11328.0,20,20,38,7.6,...,1834.0,4533.0,7692.0,2678.0,28085.0,65964.0,267.0,1007,36.6,64.2
4,1009,Blount,10908.0,NaN,5620.0,11336.0,21,20,34,8.5,...,4641.0,11256.0,17731.0,3855.0,56300.0,144559.0,267.0,1009,31.9,56.6


In [183]:
# df[['cases_2020', 'cases_2021', 'cases_2022', 'deaths_2020', 'deaths_2021', 'deaths_2022', 'Population']]

In [370]:
# Drop rows that we will not be using 'Segregation index black/white',
df.drop(columns = ['County', 'YPLL Rate (Black)', 'YPLL Rate (Hispanic)', 'YPLL Rate (White)', 'Number Uninsured', 'Number Primary Care Physicians', 'FIPS_y', 'Number pre-mature Deaths',
                        'Preventable Hosp. Rate (Black)', 'Preventable Hosp. Rate (Hispanic)', 'Preventable Hosp. Rate (White)',  'Percent Vaccinated Flu (Black)', 'Percent Uninsured',
                        'Percent  Vaccinated (Hispanic) Flu', 'Percent Vaccinated (White) Flu', 'Number Some College', 'Number Unemployed', 'Labor Force', 'PCP Ratio', 
                        '80th Percentile Income', '20th Percentile Income', '95% CI - Low', '95% CI - High', 'Life Expectancy (Black)', 'Life Expectancy (Hispanic)', 
                        'Life Expectancy (White)', 'Number HIV Cases', 'Household income (Black)', 'Household income (Hispanic)', 'Household income (White)'], inplace = True)

In [371]:
df.columns
# df.dtypes
# df.shape

Index(['FIPS_x', 'Years of Potential Life Lost Rate (premature death)',
       '% Fair/Poor Health', 'percent_smokers', 'percent_obese',
       'Food Environment Index', '% Physically Inactive',
       'percent Excessive Drinking', 'PCP Rate', 'Preventable Hosp stays Rate',
       'Percent Vaccinated Flu', 'High School Graduation Rate',
       'Percent Some College', 'Percent Unemployed', 'Income Ratio',
       'Average Daily PM2.5', 'Presence of water violation',
       'Percent Severe Housing Problems', 'Severe Housing Cost Burden',
       'Overcrowding', 'Inadequate Facilities', 'Life Expectancy',
       'Percent Diabetic', 'HIV Prevalence Rate', 'Percent Food Insecure',
       'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Percent Uninsured children', 'Median Household Income',
       'Segregation index black/white', 'Segregation Index non-white/ white',
       'Population', 'percent < 18', 'percent 65 and over',
       'percent African American', 'percent Americ

In [372]:
# Make FIPS index 
df.set_index('FIPS_x', inplace=True)

In [89]:
# # Create new columns for per populaltion stats - YPL, Number pre-mature Deaths, Number rural 
# df['YPL'] = df['Years of Potential Life Lost Rate (premature death)']/df['Population']
# df['pre mature deaths'] = df['Number pre-mature Deaths']/df['Population']
# df['rural'] = df['number Rural']/df['Population']

# df.drop(columns = ['Years of Potential Life Lost Rate (premature death)', 'Number pre-mature Deaths', 'number Rural'], inplace = True)

In [373]:
# Dummify State and Presence of water violation
# df = pd.get_dummies(columns = ['State'], data = df, drop_first=True)
df['water'] = df['Presence of water violation'].map({'No': 0, 'Yes': 1})
df.drop(columns = ['Presence of water violation', 'State'], inplace = True)

In [374]:
# Calculate total cases and deaths, and convert to % of population 
df['cases'] = df['cases_2022']
df['deaths'] = df['deaths_2022']

# df['case_rate'] = df['cases']/df['Population']
# df['death_rate'] = df['deaths']/df['Population']

# Deaths seem to be off? More deaths than population 
df.drop(columns = ['cases_2020', 'cases_2021', 'cases_2022', 'deaths_2020', 'deaths_2021', 'deaths_2022'], inplace = True)

In [375]:
# Drop Na values (3k rows -> 1828)
df.dropna(inplace=True)
df.shape

(1828, 47)

## Linear Regression (LR)

In [376]:
y = df['cases']
# X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
#        'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
X = df.drop(columns = ['cases', 'deaths'])
ols = sm.OLS(y, X).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  cases   R-squared (uncentered):                   0.963
Model:                            OLS   Adj. R-squared (uncentered):              0.962
Method:                 Least Squares   F-statistic:                              1022.
Date:                Mon, 06 Nov 2023   Prob (F-statistic):                        0.00
Time:                        18:46:09   Log-Likelihood:                         -21258.
No. Observations:                1828   AIC:                                  4.261e+04
Df Residuals:                    1783   BIC:                                  4.285e+04
Df Model:                          45                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================================================
                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
Years of Potential Life Lost Rate (premature death)    -0.8611      0.912     -0.944      0.345      -2.650       0.927
% Fair/Poor Health                                   -170.1889    509.162     -0.334      0.738   -1168.806     828.429
percent_smokers                                      -492.0777    483.051     -1.019      0.308   -1439.483     455.328
percent_obese                                         -54.1183    258.860     -0.209      0.834    -561.819     453.582
Food Environment Index                                827.8671   1632.478      0.507      0.612   -2373.904    4029.638
% Physically Inactive                                 741.3696    257.681      2.877      0.004     235.980    1246.759
percent Excessive Drinking                           -719.3080    358.283     -2.008      0.045   -1422.006     -16.610
PCP Rate                                              -17.1265     29.452     -0.582      0.561     -74.890      40.637
Preventable Hosp stays Rate                             0.6566      0.553      1.187      0.235      -0.428       1.741
Percent Vaccinated Flu                               -182.9488    111.585     -1.640      0.101    -401.800      35.903
High School Graduation Rate                            -6.5022    126.275     -0.051      0.959    -254.165     241.161
Percent Some College                                  195.6582    126.886      1.542      0.123     -53.202     444.519
Percent Unemployed                                  -1240.2019    685.115     -1.810      0.070   -2583.914     103.511
Income Ratio                                          -22.6961   1443.651     -0.016      0.987   -2854.122    2808.730
Average Daily PM2.5                                 -1198.6586    513.482     -2.334      0.020   -2205.748    -191.570
Percent Severe Housing Problems                      -527.1857   1084.165     -0.486      0.627   -2653.553    1599.181
Severe Housing Cost Burden                           1135.4915   1043.295      1.088      0.277    -910.718    3181.701
Overcrowding                                          578.5426   1067.890      0.542      0.588   -1515.905    2672.991
Inadequate Facilities                                 467.4937   1142.273      0.409      0.682   -1772.840    2707.828
Life Expectancy                                       -48.0470    734.389     -0.065      0.948   -1488.401    1392.307
Percent Diabetic                                     -302.2717    594.045     -0.509      0.611   -1467.370     862.826
HIV Prevalence Rate                                     2.7970      4.926      0.568      0.570      -6.

In [377]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [378]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [379]:
print(f'Train Score: {lr.score(X_train, y_train)}')
print(f'Test Score: {lr.score(X_test, y_test)}')

Train Score: 0.9639780952509502
Test Score: 0.9324543775846392


In [380]:
pd.DataFrame(set(zip(X.columns, lr.coef_)), columns = ['Variable Name', 'Coefficient']).sort_values('Coefficient').round(1)

,Variable Name,Coefficient
5,percent Asian,-2287.5
8,Average Daily PM2.5,-1342.3
1,Percent Unemployed,-1209.2
0,Percent Uninsured Adults,-416.0
2,Masks,-4.0
6,Population,0.3
7,% Physically Inactive,416.4
4,Percent Insufficient Sleep,540.1
3,percent Not Proficient in English,1222.3


## K Nearest Neighbors (KNN)

In [381]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [382]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [383]:
knn = KNeighborsRegressor(n_neighbors=10)
knn.fit(X_train_sc, y_train)

KNeighborsRegressor(n_neighbors=10)

In [384]:
print(f'Train Score: {knn.score(X_train_sc, y_train)}')
print(f'Test Score: {knn.score(X_test_sc, y_test)}')

Train Score: 0.6883310563076002
Test Score: 0.7165832065461339


## Decision Trees (DT)

In [385]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [386]:
%%time
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

CPU times: total: 15.6 ms
Wall time: 13.3 ms


DecisionTreeRegressor()

In [387]:
print(f'Train Score: {tree.score(X_train, y_train)}')
print(f'Test Score: {tree.score(X_test, y_test)}')

Train Score: 1.0
Test Score: 0.8642313832005694


In [388]:
# Top Features
pd.DataFrame({'feature': X.columns, 'importance': tree.feature_importances_}).sort_values('importance', ascending = False)

,feature,importance
5,Population,0.989913
4,Percent Uninsured Adults,0.002987
6,percent Asian,0.001888
7,percent Not Proficient in English,0.001690
2,Average Daily PM2.5,0.001083
0,% Physically Inactive,0.001056
8,Masks,0.000567
1,Percent Unemployed,0.000450
3,Percent Insufficient Sleep,0.000366


### Bagged Decision Tree

In [292]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [293]:
pgrid = {
    'estimator__max_depth': np.append(np.arange(1, 100), None),
}

tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
bag = BaggingRegressor(tree,random_state = 42)
tree_bagged = GridSearchCV(bag, pgrid, cv = 5, n_jobs = 8)
# tree_bagged.get_params()

In [294]:
%%time
tree_bagged.fit(X_train, y_train)

CPU times: total: 266 ms
Wall time: 8.08 s


GridSearchCV(cv=5,
             estimator=BaggingRegressor(estimator=DecisionTreeRegressor(),
                                        random_state=42),
             n_jobs=8,
             param_grid={'estimator__max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, None], dtype=object)})

In [296]:
print(f'Train Score: {tree_bagged.score(X_train, y_train)}')
print(f'Test Score: {tree_bagged.score(X_test, y_test)}')

Train Score: 0.8911918973141624
Test Score: 0.8185710802178838


In [234]:
tree_bagged.best_params_

{'estimator__max_depth': 4}

## Random Forest (RF)

In [297]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [298]:
rf = RandomForestRegressor(
    random_state = 42,
)

In [299]:
%%time
rf.fit(X_train, y_train)

CPU times: total: 891 ms
Wall time: 930 ms


RandomForestRegressor(random_state=42)

In [300]:
print(f'Train Score: {rf.score(X_train, y_train)}')
print(f'Test Score: {rf.score(X_test, y_test)}')

Train Score: 0.9469735268457481
Test Score: 0.8434835697191293


In [301]:
pd.DataFrame({'Features': X.columns, 'Importance': rf.feature_importances_}).sort_values('Importance', ascending = False)

,Features,Importance
5,Population,0.981964
7,percent Not Proficient in English,0.005337
6,percent Asian,0.002583
4,Percent Uninsured Adults,0.002447
2,Average Daily PM2.5,0.002087
0,% Physically Inactive,0.001962
8,Masks,0.001439
3,Percent Insufficient Sleep,0.001220
1,Percent Unemployed,0.000962


### RF w/ RandomSearchCV

In [302]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [303]:
params = {
    'max_features': np.arange(5, X.shape[1] + 1),
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_depth': np.append(np.arange(1, 100), None),
}

# params = {
#  'bootstrap': [True, False],
#  'max_depth': np.append(np.arange(1, 100), None),
#  'max_features': ['log2', 'sqrt'],
#  'min_samples_leaf': np.arange(1, 10),
#  'min_samples_split': np.arange(1, 20),
#  'n_estimators': [50, 100, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
# }

rf = RandomForestRegressor(
    random_state = 42,
)


rf_rs = RandomizedSearchCV(rf, params, n_iter=20, cv = 5, n_jobs = 8)
# rf_rs.get_params()

In [304]:
%%time
rf_rs.fit(X_train, y_train)

CPU times: total: 891 ms
Wall time: 6.7 s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=20, n_jobs=8,
                   param_distributions={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, None], dtype=object),
                                        'max_features': array([5, 6, 7, 8, 9]),
                                        'min_samples_leaf': [1, 2, 3, 4, 5]})

In [175]:
print(f'Train Score: {rf_rs.score(X_train, y_train)}')
print(f'Test Score: {rf_rs.score(X_test, y_test)}')

Train Score: 0.9654484819767989
Test Score: 0.6378001277376578


In [ ]:
0.9089560908215213

In [79]:
rf_rs.best_params_

{'min_samples_leaf': 2, 'max_features': 28, 'max_depth': 61}

### RF w/ Gradboost

In [305]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [306]:
rf_gb = GradientBoostingRegressor()

In [307]:
%%time
rf_gb.fit(X_train, y_train)

CPU times: total: 172 ms
Wall time: 192 ms


GradientBoostingRegressor()

In [308]:
print(f'Train Score: {rf_gb.score(X_train, y_train)}')
print(f'Test Score: {rf_gb.score(X_test, y_test)}')

Train Score: 0.9973426354173612
Test Score: 0.8776229896533103


In [ ]:
0.8875985316306076

#### RF w/ Gradboost & RandomSearchCV

In [309]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [310]:
params = {
    'max_features': np.arange(5, X.shape[1] + 1),
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_depth': np.append(np.arange(1, 100), None),
}

rf_gb = GradientBoostingRegressor()


rf_gb_rs = RandomizedSearchCV(rf_gb, params, n_iter=20, cv = 5, n_jobs = 8)
# rf_gb_rs.get_params()

In [311]:
%%time
rf_gb_rs.fit(X_train, y_train)

CPU times: total: 1.41 s
Wall time: 14.4 s


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_iter=20,
                   n_jobs=8,
                   param_distributions={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, None], dtype=object),
                                        'max_features': array([5, 6, 7, 8, 9]),
                                        'min_samples_leaf': [1, 2, 3, 4, 5]})

In [312]:
print(f'Train Score: {rf_gb_rs.score(X_train, y_train)}')
print(f'Test Score: {rf_gb_rs.score(X_test, y_test)}')

Train Score: 0.9999999992944921
Test Score: 0.8504589233832689


In [313]:
rf_gb_rs.best_params_

{'min_samples_leaf': 1, 'max_features': 8, 'max_depth': 72}

In [ ]:
0.7766914379956544
0.6930124304398317

## Extra Trees (ET)

In [314]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [315]:
%%time
et = ExtraTreesRegressor()
et.fit(X_train, y_train)

CPU times: total: 469 ms
Wall time: 528 ms


ExtraTreesRegressor()

In [316]:
print(f'Train Score: {et.score(X_train, y_train)}')
print(f'Test Score: {et.score(X_test, y_test)}')

Train Score: 1.0
Test Score: 0.9112354205179171


In [ ]:
0.6883206603467942

#### ET w/ RandomSearchCV

In [317]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [318]:
params = {
    'max_features': np.arange(5, X.shape[1] + 1),
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_depth': np.append(np.arange(1, 100), None),
}


et = ExtraTreesRegressor()

et_rs = RandomizedSearchCV(et, params, n_iter=20, cv = 5, n_jobs = 8)
# et_rs.get_params()

In [319]:
%%time
et_rs.fit(X_train, y_train)

CPU times: total: 328 ms
Wall time: 3.02 s


RandomizedSearchCV(cv=5, estimator=ExtraTreesRegressor(), n_iter=20, n_jobs=8,
                   param_distributions={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, None], dtype=object),
                                        'max_features': array([5, 6, 7, 8, 9]),
                                        'min_samples_leaf': [1, 2, 3, 4, 5]})

In [320]:
print(f'Train Score: {et_rs.score(X_train, y_train)}')
print(f'Test Score: {et_rs.score(X_test, y_test)}')

Train Score: 0.89784222306703
Test Score: 0.8864894016736619


In [321]:
et_rs.best_params_

{'min_samples_leaf': 2, 'max_features': 9, 'max_depth': 44}

In [ ]:
0.9183893600656411
0.6577361119596212

## Ensembled Model (Voting Regressor)

In [322]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [323]:
vr = VotingRegressor([
    ('lr', lr),
    ('tree', tree_bagged),
    ('rf_rs', rf_rs),
    ('rf_gb', rf_gb),
    ('et_rs', et_rs)
])

In [324]:
%%time
vr.fit(X_train, y_train)

CPU times: total: 2.78 s
Wall time: 17 s


VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('tree',
                             GridSearchCV(cv=5,
                                          estimator=BaggingRegressor(estimator=DecisionTreeRegressor(),
                                                                     random_state=42),
                                          n_jobs=8,
                                          param_grid={'estimator__max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,...
                                                param_distributions={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, None], dtype=object),
                                                                     'max_features': array([5, 6, 7, 8, 9]),
                                                                     'min_samples_leaf': [1,
                                                                                          2,
                                                                                          3,
                                                                                          4,
                                                                                          5]}))])

In [325]:
print(f'Train Score: {vr.score(X_train, y_train)}')
print(f'Train Score: {vr.score(X_test, y_test)}')

Train Score: 0.9813983323826954
Train Score: 0.8770621996681867


In [ ]:
0.9067405328360387
0.8770621996681867